# 概要
- 各データの期間について
- 予測対象となるauctionについて
- テストデータについて
- どのオークションがシステム的に候補から外れるか

In [3]:
from os.path import dirname
import os
import datetime
from dateutil.relativedelta import relativedelta

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
%load_ext autoreload
%autoreload 2
import brandear_est as be

In [5]:
INPUT_DIR = os.path.join(os.getcwd(), "../../data/input/")
IMD_DIR = os.path.join(os.getcwd(), "../../data/intermediate/")

## 各データの期間について
- 20190924以降のデータが入っているテーブル
    - auction
    - brand

In [4]:
auction = pd.read_csv(INPUT_DIR + "auction.csv")
print(auction.shape)
print(auction["CreateDate"].max())
print(auction[["ShouhinID"]].nunique())
auction.head()

(4109559, 13)
2019-09-30 10:00:58
ShouhinID    3002614
dtype: int64


,AuctionID,ShouhinShubetsuID,ShouhinID,SaishuppinKaisuu,ConditionID,BrandID,GenreID,GenreGroupID,LineID,ColorID,DanjobetsuID,SankouKakaku,CreateDate
0,4066465,1,620477,334,6,494,321,312,0,15,0,4000,2019-07-27 09:31:34
1,622037,1,1146271,0,6,32,321,312,0,304,0,4500,2019-07-27 09:31:34
2,3447030,1,1069454,0,6,30,320,312,0,338,0,3000,2019-07-27 09:31:34
3,957658,1,661292,0,6,30,321,312,0,188,0,3500,2019-07-27 09:31:34
4,2496085,1,2207903,0,6,569,6,314,0,0,0,8000,2019-07-27 09:31:34


In [5]:
brand = pd.read_csv(INPUT_DIR + "brand.csv")
print(brand.shape)
print(brand["CreateDate"].max())
brand.head()

(9834, 5)
2019-10-25 06:00:01


,BrandID,BrandName1,BrandName2,ModifyDate,CreateDate
0,1,アンテプリマ,ANTEPRIMA,2010-09-17 11:25:29,2010-09-10 12:06:39
1,2,アイグナー,AIGNER,2010-09-17 11:25:29,2010-09-10 12:06:39
2,3,アルマーニジーンズ,ARMANIJEANS,2010-09-17 11:25:29,2010-09-10 12:06:39
3,4,アルマーニエクスチェンジ,ARMANIEX,2010-09-17 11:25:29,2010-09-10 12:06:39
4,5,アナスイ,ANNA SUI,2010-09-17 11:25:29,2010-09-10 12:06:39


In [6]:
brand.sort_values("ModifyDate", ascending=False).head(20)

,BrandID,BrandName1,BrandName2,ModifyDate,CreateDate
9635,10897,カリンスティーブンス,karin stevens,2019-10-25 06:00:01,2019-10-25 06:00:01
1870,2397,ミルクボーイ,MILKBOY,2019-10-25 06:00:01,2019-10-25 06:00:01
9765,11123,フォームラ ホベン,formul@JOVEN,2019-10-25 06:00:01,2019-10-25 06:00:01
9827,11218,ラルドゥート,LA REDOUTE,2019-10-25 06:00:01,2019-10-25 06:00:01
9636,10899,エスレイ,Esley,2019-10-25 06:00:01,2019-10-25 06:00:01
9762,11117,アレクシアアドマーニューヨーク,ALEXIA ADMOR NEW YORK,2019-10-23 06:00:04,2019-10-23 06:00:04
9755,11105,インスパイアド,inspired,2019-10-23 06:00:04,2019-10-23 06:00:04
9753,11103,ニンフェ,NYMPHE,2019-10-23 06:00:04,2019-10-23 06:00:04
9721,11048,メイドフォーインパルスファッションウィーク,MADE for impulse FASHION WEEK,2019-10-23 06:00:04,2019-10-23 06:00:04
9720,11047,タルト,TART,2019-10-23 06:00:04,2019-10-23 06:00:04


In [14]:
category = pd.read_csv(INPUT_DIR + "category.csv")
print(category["CreateDate"].max())
print(category.describe())
category

2010-09-15 14:55:36
       CategoryID
count   12.000000
mean     6.500000
std      3.605551
min      1.000000
25%      3.750000
50%      6.500000
75%      9.250000
max     12.000000


,CategoryID,CategoryName,ModifyDate,CreateDate
0,1,バッグ,2010-09-15 14:55:36,2010-09-15 14:55:36
1,2,財布,2010-09-15 14:55:36,2010-09-15 14:55:36
2,3,服飾雑貨,2010-09-15 14:55:36,2010-09-15 14:55:36
3,4,アクセサリー,2010-09-15 14:55:36,2010-09-15 14:55:36
4,5,トップス,2010-09-15 14:55:36,2010-09-15 14:55:36
5,6,ワンピース,2010-09-15 14:55:36,2010-09-15 14:55:36
6,7,ボトムス,2010-09-15 14:55:36,2010-09-15 14:55:36
7,8,アウター,2010-09-15 14:55:36,2010-09-15 14:55:36
8,9,スーツ,2010-09-15 14:55:36,2010-09-15 14:55:36
9,10,靴,2010-09-15 14:55:36,2010-09-15 14:55:36


In [8]:
color = pd.read_csv(INPUT_DIR + "color.csv")
print(color["CreateDate"].max())
color.head()

2010-04-02 13:25:07


,ColorID,ColorName,ItemColorID,ModifyDate,CreateDate
0,1,オフホワイト,1,2010-04-02 13:25:06,2010-04-02 13:25:06
1,2,オフホワイト基調,1,2010-04-02 13:25:06,2010-04-02 13:25:06
2,3,アズール,1,2010-04-02 13:25:06,2010-04-02 13:25:06
3,4,パール,1,2010-04-02 13:25:06,2010-04-02 13:25:06
4,5,パールホワイト,1,2010-04-02 13:25:06,2010-04-02 13:25:06


In [9]:
danjobetsu = pd.read_csv(INPUT_DIR + "danjobetsu.csv")
print(danjobetsu["CreateDate"].max())
danjobetsu

2010-04-02 13:25:07


,DanjobetsuID,DanjobetsuName,ModifyDate,CreateDate
0,1,メンズ,2010-04-02 13:25:07,2010-04-02 13:25:07
1,2,レディース,2010-04-02 13:25:07,2010-04-02 13:25:07
2,3,ユニセックス,2010-04-02 13:25:07,2010-04-02 13:25:07


In [13]:
genre = pd.read_csv(INPUT_DIR + "genre.csv")
print(genre["CreateDate"].max())
print(genre.describe())
genre.head()

2010-09-07 00:00:00
          GenreID  GenreGroupID  ItemShouID  CategoryID
count  101.000000    101.000000  101.000000  101.000000
mean   205.584158    142.564356   33.039604    4.772277
std    140.499343    136.890644   18.905513    3.016227
min      1.000000      1.000000    1.000000    0.000000
25%     38.000000     21.000000   14.000000    2.000000
50%    205.000000     35.000000   35.000000    4.000000
75%    340.000000    312.000000   51.000000    8.000000
max    365.000000    314.000000   65.000000   12.000000


,GenreID,GenreName,GenreGroupID,ItemShouID,CategoryID,ModifyDate,CreateDate
0,1,その他アクセサリー,1,36,4,2010-09-15 15:29:28,2010-04-02 13:25:06
1,2,アンサンブル,21,43,5,2010-09-15 15:29:28,2010-04-02 13:25:06
2,3,イヤリング,296,32,4,2010-09-15 15:29:28,2010-04-02 13:25:06
3,6,キーケース,314,12,2,2010-09-15 15:29:28,2010-04-02 13:25:06
4,7,キーホルダー,7,12,2,2010-09-15 15:29:28,2010-04-02 13:25:06


In [12]:
genregroup = pd.read_csv(INPUT_DIR + "genregroup.csv")
print(genregroup["CreateDate"].max())
print(genregroup.describe())
genregroup.head()

2010-04-02 13:25:06
       GenreGroupID  ShouhinShubetsuID
count     34.000000          34.000000
mean      99.823529           1.176471
std      110.407090           1.028992
min        1.000000           1.000000
25%       22.500000           1.000000
50%       34.500000           1.000000
75%      179.750000           1.000000
max      314.000000           7.000000


,GenreGroupID,GenreGroupName,ShouhinShubetsuID,ModifyDate,CreateDate
0,1,アクセサリー（他）,1,2010-04-02 13:25:06,2010-04-02 13:25:06
1,7,キーホルダー,1,2010-04-02 13:25:06,2010-04-02 13:25:06
2,9,コート,1,2010-04-02 13:25:06,2010-04-02 13:25:06
3,10,サングラス,1,2010-04-02 13:25:06,2010-04-02 13:25:06
4,13,ジャケット,1,2010-04-02 13:25:06,2010-04-02 13:25:06


In [6]:
itemshou = pd.read_csv(INPUT_DIR + "itemshou.csv")
print(itemshou["CreateDate"].max())
itemshou.head()

2010-09-07 00:00:00


,ItemShouID,ItemShouName,ItemDaiID,ModifyDate,CreateDate
0,1,ハンドバッグ,1,2010-04-02 13:25:06,2010-04-02 13:25:06
1,2,ショルダーバッグ,1,2010-04-02 13:25:06,2010-04-02 13:25:06
2,3,トートバッグ,1,2010-04-02 13:25:06,2010-04-02 13:25:06
3,4,セカンド・クラッチバッグ,1,2010-04-02 13:25:06,2010-04-02 13:25:06
4,5,リュックサック,1,2010-04-02 13:25:06,2010-04-02 13:25:06


In [7]:
itemshou.describe()

,ItemShouID,ItemDaiID
count,65.00000,65.000000
mean,33.00000,3.523077
std,18.90767,2.165841
min,1.00000,1.000000
25%,17.00000,2.000000
50%,33.00000,3.000000
75%,49.00000,4.000000
max,65.00000,9.000000


In [4]:
line = pd.read_csv(INPUT_DIR + "line.csv")
print(line["CreateDate"].max())
line.head()

2019-08-06 11:17:07


,LineID,LineName,BrandID,ItemLineID,ModifyDate,CreateDate
0,1,エピ,114,7,2010-04-02 13:25:06,2010-04-02 13:25:06
1,2,タイガ,114,6,2010-04-02 13:25:06,2010-04-02 13:25:06
2,3,ダミエ,114,5,2010-04-02 13:25:06,2010-04-02 13:25:06
3,4,モノグラムマルチカラー,114,3,2010-04-02 13:25:06,2010-04-02 13:25:06
4,5,モノグラム,114,1,2010-04-02 13:25:06,2010-04-02 13:25:06


In [5]:
line.describe()

,LineID,BrandID,ItemLineID
count,267.000000,267.0,267.000000
mean,134.000000,114.0,6.636704
std,77.220464,0.0,2.739135
min,1.000000,114.0,1.000000
25%,67.500000,114.0,8.000000
50%,134.000000,114.0,8.000000
75%,200.500000,114.0,8.000000
max,267.000000,114.0,8.000000


In [460]:
line.sort_values("ModifyDate", ascending=False).head()

,LineID,LineName,BrandID,ItemLineID,ModifyDate,CreateDate
133,134,アンティア,114,8,2019-08-06 11:17:07,2019-08-06 11:17:07
183,184,ダミエパイエット,114,8,2019-08-06 11:17:07,2019-08-06 11:17:07
169,170,パルナセア,114,8,2019-08-06 11:17:07,2019-08-06 11:17:07
170,171,スウィートモノグラム,114,8,2019-08-06 11:17:07,2019-08-06 11:17:07
171,172,モノグラムアップリケ,114,8,2019-08-06 11:17:07,2019-08-06 11:17:07


In [6]:
kaiin = pd.read_csv(INPUT_DIR + "kaiin.csv")
print(kaiin["CreateDate"].max())
kaiin.head()

2019-10-25 16:05:20


,KaiinID,RepeaterFlag,SeinengappiDate,CreateDate
0,171036,1.0,NaN,2010-11-08 06:38:50
1,160541,0.0,NaN,2010-11-08 06:51:10
2,271092,NaN,NaN,2010-11-08 08:11:57
3,124089,NaN,NaN,2010-11-08 09:20:56
4,256950,NaN,NaN,2010-11-08 09:33:51


In [9]:
print(kaiin["RepeaterFlag"].value_counts())
print(kaiin.shape[0])
print(kaiin[kaiin["SeinengappiDate"].isnull()==False].shape[0])
kaiin[kaiin["SeinengappiDate"].isnull()==False].head()

0.0    44476
1.0    37186
Name: RepeaterFlag, dtype: int64
277894
2106


,KaiinID,RepeaterFlag,SeinengappiDate,CreateDate
34,271260,1.0,1981-08-04,2010-11-09 00:35:02
65,89068,1.0,1966-01-10,2010-11-09 14:33:16
107,38900,1.0,1970-10-07,2010-11-09 22:05:24
231,245435,1.0,1981-05-17,2010-11-11 13:48:26
488,136704,1.0,1964-07-20,2010-11-14 20:59:34


In [15]:
watchlist = pd.read_csv(INPUT_DIR + "watchlist.csv")
print(watchlist.shape)
print(watchlist["SakujoFlag"].value_counts())
print(watchlist["TourokuDate"].max())
watchlist.head()

(4165155, 4)
1    3921759
0     243396
Name: SakujoFlag, dtype: int64
2019-09-23 23:59:59


,KaiinID,AuctionID,TourokuDate,SakujoFlag
0,231713,1036190,2018-09-01 00:00:16,1
1,225585,4043640,2018-09-01 00:00:34,1
2,75217,3100882,2018-09-01 00:00:44,1
3,86693,655600,2018-09-01 00:00:51,1
4,267234,2607395,2018-09-01 00:01:09,1


In [16]:
shudounyuusatsu = pd.read_csv(INPUT_DIR + "shudounyuusatsu.csv")
print(shudounyuusatsu.shape)
print(shudounyuusatsu["SakujoFlag"].value_counts())
print(shudounyuusatsu["ShudouNyuusatsuDate"].max())
shudounyuusatsu.head()

(612516, 7)
1    612516
Name: SakujoFlag, dtype: int64
2019-09-23 23:59:09


,AuctionID,KaiinID,ShudouNyuusatsuDate,Kingaku,Suuryou,SokketsuFlag,SakujoFlag
0,3744587,227198,2018-09-01 21:40:44,18000,1,0,1
1,636604,116853,2018-09-01 18:56:28,20000,1,0,1
2,636604,258477,2018-09-01 19:46:13,20500,1,0,1
3,636604,116853,2018-09-01 19:54:17,22000,1,0,1
4,636604,258477,2018-09-01 20:47:49,21500,1,0,1


In [17]:
rakusatsu = pd.read_csv(INPUT_DIR + "rakusatsu.csv")
print(rakusatsu.shape)
print(rakusatsu["SakujoFlag"].value_counts())
print(rakusatsu["RakusatsuDate"].max())
rakusatsu.head()

(450242, 6)
0    450229
1        13
Name: SakujoFlag, dtype: int64
2019-09-23 23:59:52


,AuctionID,KaiinID,RakusatsuDate,Kingaku,Suuryou,SakujoFlag
0,3744587,227198,2018-09-01 21:50:00,18000,1,0
1,3150550,253112,2018-09-01 21:07:00,16500,1,0
2,636604,258477,2018-09-01 22:43:00,24500,1,0
3,2413186,227198,2018-09-01 21:31:00,26700,1,0
4,2786032,69415,2018-09-01 22:19:00,128905,1,0


In [18]:
nyuuka_oshirase = pd.read_csv(INPUT_DIR + "nyuuka_oshirase.csv")
print(nyuuka_oshirase["CreateDate"].max())
nyuuka_oshirase.head()

2019-09-23 23:56:18


,KaiinID,BrandID,CategoryID,GenreID,KaishikakakuLow,KaishikakakuHigh,ItemColorID,ConditionID,conditionFrom,conditionTo,SearchFullText,Size,MailSendFlag,CreateDate
0,273964,9712.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2019-09-01 00:06:25
1,64255,283.0,NaN,NaN,NaN,2000.0,NaN,NaN,7.0,4.0,NaN,NaN,0,2019-09-01 00:10:20
2,274994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ファー,NaN,0,2019-09-01 00:14:43
3,132156,7516.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2019-09-01 00:23:38
4,132156,1698.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2019-09-01 00:23:38


In [19]:
search_log = pd.read_csv(INPUT_DIR + "search_log.tsv", sep="\t")
print(search_log["TourokuTime"].max())
search_log.head()

/Users/nobuyuki.ishida/.pyenv/versions/3.7.5/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


2019-09-23 23:59:59


,SearchFullText,KensakuJouken,KensakuKekkaKenSuu,TourokuTime,Device,KaiinID,BrandID,CategoryID,ShinchakuFlag,ConditionID,Size,DanjobetsuID,Sozai,ColorID,PriceLow,PriceHigh
0,エクスプローラー,BrandID=118&SearchFullText=エクスプローラー&ItemOrder=...,0.0,2019-09-01 00:00:01,sp,NaN,118.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ボッテガヴェネタ,SearchFullText=ボッテガヴェネタ&ItemOrder=0&ItemDispla...,1596.0,2019-09-01 00:00:03,sp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,マンハッタンポーテージ,SearchFullText=マンハッタンポーテージ&ItemOrder=1&ItemDis...,66.0,2019-09-01 00:00:04,sp,102997.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,マンハッタンポーテージ,SearchFullText=マンハッタンポーテージ&ItemOrder=1&ItemDis...,66.0,2019-09-01 00:00:04,sp,102997.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11.5メンズ,BrandID=2010&SearchFullText=11.5メンズ&ItemOrder=...,0.0,2019-09-01 00:00:04,sp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 予測対象となるauctionについて

- 一週間で大体6000ユーザー、7万オークション、10万ペアくらい
- 予測対象となるオークションの60%程度が201907以降のオークション
    - 201907以降で約90万オークション
- 予測対象となるオークションの80%程度が201907以降にwatchされている？
- watchlistのアクション直近3月、オークション作成直近3月で絞ってもほとんどカバーできている様子
- 一月あたり30万件程度のオークションが新規に登録されている
- 一週間にアクションを起こすユーザーは大体6000くらい
    - 学習データのレコードを1億程度にしようと思うと一人当たり16000レコードくらいにしなきゃいけない？

In [406]:
auction = pd.read_csv(INPUT_DIR + "auction.csv")
watchlist = pd.read_csv(INPUT_DIR + "watchlist.csv")
shudounyuusatsu = pd.read_csv(INPUT_DIR + "shudounyuusatsu.csv")

In [407]:
auction = be.to_datetime(auction)
watchlist = be.to_datetime(watchlist)
shudounyuusatsu = be.to_datetime(shudounyuusatsu)
watchlist.rename(columns={"TourokuDate": "ActionDate"}, inplace=True)
shudounyuusatsu.rename(columns={"ShudouNyuusatsuDate": "ActionDate"}, inplace=True)
watch_nyuusatsu = pd.concat([watchlist, shudounyuusatsu], sort=False)

In [414]:
# 対象期間中にアクションのあったデータ
start_dt = datetime.datetime(2019, 9, 17, 0, 0, 0)
end_dt = datetime.datetime(2019, 9, 24, 0, 0, 0)
month_delta = 3
target_log = watch_nyuusatsu[(watch_nyuusatsu["ActionDate"] >= start_dt) & (watch_nyuusatsu["ActionDate"] <= end_dt)]

target_auctions = (
    target_log[["AuctionID"]]
    .drop_duplicates()
    .merge(auction, on="AuctionID", how="inner")
)

print(target_log[["KaiinID"]].nunique())
print(target_log[["AuctionID"]].nunique())
print(target_log.shape)

KaiinID    6344
dtype: int64
AuctionID    68682
dtype: int64
(97583, 7)


In [415]:
target_log[["KaiinID", "AuctionID"]].drop_duplicates().shape

(91044, 2)

In [416]:
def extract_ym(df, datecol):
    return  getattr(df[datecol].dt, "year").astype(str) + getattr(df[datecol].dt, "month").astype(str).str.zfill(2)

In [417]:
# 対象期間中にアクションのあったオークション
#  3月くらいまでで97%くらいカバーできている
target_auctions["CreateYearMonth"] = extract_ym(target_auctions, "CreateDate")
print(target_auctions.shape)
print(target_auctions[["ShouhinID"]].nunique())
print(target_auctions["CreateYearMonth"].value_counts())
print(target_auctions["CreateYearMonth"].value_counts().cumsum() / target_auctions.shape[0])

(68682, 14)
ShouhinID    68430
dtype: int64
201909    26033
201908    17360
201907    12991
201906     5381
201905     2788
201904     1421
201903      945
201901      379
201902      369
201811      326
201812      277
201810      265
201809       84
201808       63
Name: CreateYearMonth, dtype: int64
201909    0.379037
201908    0.631796
201907    0.820943
201906    0.899289
201905    0.939882
201904    0.960572
201903    0.974331
201901    0.979849
201902    0.985222
201811    0.989968
201812    0.994001
201810    0.997860
201809    0.999083
201808    1.000000
Name: CreateYearMonth, dtype: float64


In [418]:
# 対象期間中にアクションのあったオークションの、対象期間以前のお気に入り登録時期
# 半数以上はお気に入り履歴がなく、あるものは直近ばかり
target_past_watch  = (
    target_auctions[["AuctionID", "CreateDate"]].merge(
        watchlist[watchlist["ActionDate"] <= start_dt],
        on="AuctionID", how="inner")
)
target_past_watch["ActionYearMonth"] = extract_ym(target_past_watch, "ActionDate")
print("予測対象期間以前にお気に入り登録されているオークション割合", target_past_watch["AuctionID"].nunique() / target_auctions.shape[0])
print(target_past_watch[["AuctionID", "ActionYearMonth"]].groupby("AuctionID").max()["ActionYearMonth"].value_counts())
print(target_past_watch[["AuctionID", "ActionYearMonth"]].groupby("AuctionID").max()["ActionYearMonth"].value_counts().cumsum()\
      / target_past_watch["AuctionID"].nunique())

予測対象期間以前にお気に入り登録されているオークション割合 0.5411461518301739
201909    22989
201908     8751
201907     2968
201906     1387
201905      461
201904      272
201903      165
201902       82
201901       49
201812       27
201811       11
201810        5
Name: ActionYearMonth, dtype: int64
201909    0.618533
201908    0.853983
201907    0.933839
201906    0.971157
201905    0.983561
201904    0.990879
201903    0.995318
201902    0.997525
201901    0.998843
201812    0.999570
201811    0.999865
201810    1.000000
Name: ActionYearMonth, dtype: float64


In [449]:
# 実際に候補を作成した際、どのくらいの正解データペアが候補内に入っているかを調べる
auc_cand = be.left_anti_join(
    pd.concat([
    auction[(auction["CreateDate"] > start_dt - relativedelta(months=2)) &
             (auction["CreateDate"] < end_dt)][["AuctionID"]],
    watchlist[(watchlist["ActionDate"] > start_dt - relativedelta(months=2)) &
             (watchlist["ActionDate"] < start_dt) ][["AuctionID"]].drop_duplicates()
    ]).drop_duplicates(),
    rakusatsu, "AuctionID", "AuctionID"
)
print(auc_cand.shape)
print(auc_cand.shape[0] * 6000)

(742949, 1)
4457694000


In [450]:
print("カバー割合 : ", target_auctions.merge(auc_cand, on="AuctionID", how="inner").shape[0] / target_auctions.shape[0])

カバー割合 :  0.7871785911883754


## テストデータについて

- 予測対象ユーザーは6016

In [137]:
sample_sub = pd.read_csv(INPUT_DIR + "sample_submission.csv")

In [139]:
sample_sub[["KaiinID"]].nunique()

KaiinID    6016
dtype: int64

## どのオークションがシステム的に候補から外れるか

- 入札に含まれているオークションは落札とほぼ一致している

### 結論
- 落札に含まれているオークションは終了していると見なして除外する
- 他にもシステム的に候補に入っていないオークションが残っている可能性が高い(watchから確認できるかもしれない)

In [181]:
auction.shape

(4109559, 13)

In [187]:
print(rakusatsu.shape[0])
print(rakusatsu[["AuctionID"]].drop_duplicates().shape[0])

450242
450205


In [195]:
print(watchlist.shape[0])
print(watchlist[["AuctionID"]].drop_duplicates().shape[0])

4165155
1484732


In [196]:
print(shudounyuusatsu.shape[0])
print(shudounyuusatsu[["AuctionID"]].drop_duplicates().shape[0])

612516
453391


In [177]:
rakusatsu[["AuctionID"]].drop_duplicates().shape[0] == rakusatsu[["AuctionID"]].drop_duplicates().merge(auction, on="AuctionID").shape[0]

True

In [178]:
shudounyuusatsu[["AuctionID"]].drop_duplicates().shape[0] == shudounyuusatsu[["AuctionID"]].drop_duplicates().merge(auction, on="AuctionID").shape[0]

True

In [179]:
watchlist[["AuctionID"]].drop_duplicates().shape[0] == watchlist[["AuctionID"]].drop_duplicates().merge(auction, on="AuctionID").shape[0]

True

In [190]:
nyuusatsu_after_rakusatsu = (
    rakusatsu[["AuctionID", "RakusatsuDate"]]
    .drop_duplicates().merge(shudounyuusatsu, on="AuctionID")
    .query("RakusatsuDate <= ShudouNyuusatsuDate")
)

In [191]:
nyuusatsu_after_rakusatsu[["AuctionID"]].drop_duplicates().shape

(15418, 1)

In [197]:
rakusatsu[["AuctionID"]].drop_duplicates().merge(shudounyuusatsu[["AuctionID"]].drop_duplicates(), on="AuctionID").shape

(450138, 1)

In [193]:
watch_after_rakusatsu = (
    rakusatsu[["AuctionID", "RakusatsuDate"]]
    .drop_duplicates().merge(watchlist, on="AuctionID")
    .query("RakusatsuDate <= TourokuDate")
)

In [194]:
watch_after_rakusatsu[["AuctionID"]].drop_duplicates().shape

(5748, 1)

## お気に入りデータの削除フラグなどからオークション候補を作成できるか調べる

In [261]:
all_rmd_watch = watchlist[(watchlist["SakujoFlag"] == 1) &(watchlist["ActionDate"] < datetime.datetime(2019, 9, 1, 0, 0, 0))]

In [262]:
watch_rmd_cand = be.left_anti_join(all_rmd_watch, rakusatsu, "AuctionID", "AuctionID")

In [263]:
watch_rmd_cand["ActionYearMonth"] = extract_ym(watch_rmd_cand, "ActionDate")
print("お気に入り削除履歴から求めた、オークション候補から削除予定の候補数 : ", watch_rmd_cand.shape[0])
print(watch_rmd_cand["ActionYearMonth"].value_counts().sort_index())
print(watch_rmd_cand["ActionYearMonth"].value_counts().sort_index().cumsum() / watch_rmd_cand.shape[0])

お気に入り削除履歴から求めた、オークション候補から削除予定の候補数 :  2347344
201809    173589
201810    206741
201811    170127
201812    185630
201901    223522
201902    229411
201903    222482
201904    176467
201905    192880
201906    205883
201907    198045
201908    162567
Name: ActionYearMonth, dtype: int64
201809    0.073951
201810    0.162026
201811    0.234502
201812    0.313583
201901    0.408806
201902    0.506538
201903    0.601319
201904    0.676496
201905    0.758666
201906    0.846374
201907    0.930744
201908    1.000000
Name: ActionYearMonth, dtype: float64


In [284]:
print("謝って消去してしまう正解データ割合 : ", 
      1 - be.left_anti_join(target_auctions, watch_rmd_cand, "AuctionID", "AuctionID").shape[0] / target_auctions.shape[0])

謝って消去してしまう正解データ割合 :  0.19230657231880255


In [285]:
print("消去後のオークション候補件数 : ", be.left_anti_join(auc_cand, watch_rmd_cand, "AuctionID", "AuctionID").shape[0])

消去後のオークション候補件数 :  900229


In [401]:
100000 / (900229 * 6000)

1.8513807782982628e-05

## 中国人コードでどれくらいの正解データが残っているのか調べる

- 正解データの10%を130万件のデータでカバーしている状態

In [288]:
train_dataset = pd.read_pickle(IMD_DIR + "202001152052" + "_train_dataset.pkl")
valid_for_train_dataset = pd.read_pickle(IMD_DIR + "202001152052" + "_valid_for_train_dataset.pkl")
valid_for_sub_dataset = pd.read_pickle(IMD_DIR + "202001152055" + "_valid_for_sub_dataset.pkl")

In [294]:
dset_to_period = {
    "train": {"oldest": datetime.datetime(2019, 9, 3, 0, 0, 0),
              "newest": datetime.datetime(2019, 9, 10, 0, 0, 0)},
    "valid_for_train": {"oldest": datetime.datetime(2019, 9, 10, 0, 0, 0),
              "newest": datetime.datetime(2019, 9, 17, 0, 0, 0),},
    "valid_for_sub": {"oldest": datetime.datetime(2019, 9, 17, 0, 0, 0),
              "newest": datetime.datetime(2019, 9, 24, 0, 0, 0),}
}

In [391]:
def extract_target_aucs(watch, bid, dset_to_period, dset_type):

    oldest_dtime, newest_dtime = (
        dset_to_period[dset_type]["oldest"],
        dset_to_period[dset_type]["newest"]
    )    
    watch_actioned = (
        watch[(watch["ActionDate"] >= oldest_dtime) & (watch["ActionDate"] < newest_dtime)]
    )
    bid_actioned = (
        bid[(bid["ActionDate"] >= oldest_dtime) & (bid["ActionDate"] < newest_dtime)]
    )
    target_aucs = (
        pd.concat([watch_actioned, bid_actioned], sort=False)
    )
    return target_aucs

In [384]:
# 正解データのオークションをどのくらい含められているか
print(train_dataset.query("(watch_actioned == 1) | (bid_actioned == 1)")["AuctionID"].nunique() / \
     extract_target_aucs(watchlist, shudounyuusatsu, dset_to_period, "train")[["AuctionID"]].drop_duplicates().shape[0])

0.0948067148743767


In [398]:
# どのくらいの正解データを含んでいるか
print(train_dataset.query("(watch_actioned == 1) | (bid_actioned == 1)")[["AuctionID", "KaiinID"]].drop_duplicates().shape[0])
# 正解のペアの割合
print(train_dataset.query("(watch_actioned == 1) | (bid_actioned == 1)")[["AuctionID", "KaiinID"]].drop_duplicates().shape[0] / \
     train_dataset[["AuctionID", "KaiinID"]].drop_duplicates().shape[0])

6347
0.0019329047260997563
